# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sys
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
import re
import os
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sqlalchemy import create_engine

import pickle
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.datasets import make_multilabel_classification
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [5]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName',engine)


X = df['message']
Y = df.iloc[:,4:]
columns = Y.columns

### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    '''
    text : the text you want to tokenize
    '''
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Replace url in case we find some     
    detected_urls = re.findall(url_regex, text)
    
    # Tokenize the text     
    tokens = word_tokenize(text)
    
    #lemmatize and normalize the data     
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    Starting Verb Extractor class
    
    This class extract the starting verb of a sentence,
    creating a new feature for the ML classifier
    """

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    # Given it is a tranformer we can return the self 
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [14]:
classifier = RandomForestClassifier(min_samples_split = 100,min_samples_leaf = 20, max_depth = 8,
                                       max_features = 'sqrt', random_state = 1)
pipeline = Pipeline([

                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer()),
                ('classifier', MultiOutputClassifier(classifier))
    ])


cv = GridSearchCV(pipeline, param_grid=params, cv=5, n_jobs=-1)


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
cv.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('count_vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
 ...1,
            oob_score=False, random_state=1, verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'classifier__estimator__n_estimators': [100, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [17]:
y_pred = cv.predict(X_test)
print(classification_report(y_test.values, y_pred, target_names = columns))

                        precision    recall  f1-score   support

               related       0.77      1.00      0.87      5008
               request       0.00      0.00      0.00      1116
                 offer       0.00      0.00      0.00        23
           aid_related       0.95      0.03      0.06      2720
          medical_help       0.00      0.00      0.00       513
      medical_products       0.00      0.00      0.00       317
     search_and_rescue       0.00      0.00      0.00       181
              security       0.00      0.00      0.00       119
              military       0.00      0.00      0.00       195
                 water       0.00      0.00      0.00       402
                  food       0.00      0.00      0.00       751
               shelter       0.00      0.00      0.00       621
              clothing       0.00      0.00      0.00       104
                 money       0.00      0.00      0.00       140
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [18]:
classifier = RandomForestClassifier(min_samples_split = 100,min_samples_leaf = 20, max_depth = 8,
                                       max_features = 'sqrt', random_state = 1)
pipeline2 = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ])),

            ('starting_verb_transformer', StartingVerbExtractor())
        ])),

        ('classifier', MultiOutputClassifier(classifier))
    ])
    
params2 = {
    
            'classifier__estimator__n_estimators': [100, 200]
#     These parameters were commented because they take too long to run
#         ,
        #             'classifier__estimator__random_state' : [1,5,10],
#             'classifier__estimator__min_samples_split':[100,200,300]
        
                                                        }
cv2 = GridSearchCV(pipeline2, param_grid = params2, cv=5, n_jobs=-1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
cv2.fit(X_train, y_train)
y_pred2 = cv2.predict(X_test)
print(classification_report(y_test.values, y_pred2, target_names = columns))

                        precision    recall  f1-score   support

               related       0.77      1.00      0.87      5008
               request       0.00      0.00      0.00      1116
                 offer       0.00      0.00      0.00        23
           aid_related       0.95      0.03      0.06      2720
          medical_help       0.00      0.00      0.00       513
      medical_products       0.00      0.00      0.00       317
     search_and_rescue       0.00      0.00      0.00       181
              security       0.00      0.00      0.00       119
              military       0.00      0.00      0.00       195
                 water       0.00      0.00      0.00       402
                  food       0.00      0.00      0.00       751
               shelter       0.00      0.00      0.00       621
              clothing       0.00      0.00      0.00       104
                 money       0.00      0.00      0.00       140
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [21]:
pickle.dump(cv2, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.